In [1]:
from unlabeled_extrapolation.models import bit_resnet, vit_model, timm_model, clip_model, imnet_resnet
from unlabeled_extrapolation.utils import utils
import importlib
import timm
import torch
from torch import nn
import numpy as np
importlib.reload(bit_resnet)
importlib.reload(vit_model)
importlib.reload(utils)
importlib.reload(timm_model)
importlib.reload(clip_model)
importlib.reload(imnet_resnet)

<module 'unlabeled_extrapolation.models.imnet_resnet' from '/juice/scr/ananya/cifar_experiments/transfer_learning/unlabeled_extrapolation/models/imnet_resnet.py'>

In [2]:
import sys
print(sys.path)

['/juice/scr/ananya/cifar_experiments/transfer_learning/scripts', '/juice/scr/ananya/cifar_experiments/transfer_learning/scripts', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python38.zip', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/lib-dynload', '', '/sailhome/ananya/.local/lib/python3.8/site-packages', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/apex-0.1-py3.8-linux-x86_64.egg', '/juice/scr/ananya/cifar_experiments/wilds', '/juice/scr/ananya/cifar_experiments/transfer_learning', '/juice/scr/ananya/verified_calibration', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/IPython/extensions', '/sailhome/ananya/.ipython']


In [3]:
def get_layers_freeze_test(model):
    print('num params before freezing: ', utils.count_parameters(model, trainable=True))
    print(model.get_layers()[1])
    print(len(model.get_layers()))
    for k in [1, 2, len(model.get_layers())]:
        model.freeze_bottom_k(k=k)
        print(f'num params after freezing {k}: {utils.count_parameters(model, trainable=True)}')


# Get image

# Analyzing layer norm

In [9]:
data = np.array([0.1, 0.4, 0.5], dtype=np.float32)+0.6
x = torch.tensor(data)
layer_norm = nn.LayerNorm(len(data), elementwise_affine=False)
A = torch.eye(len(data), requires_grad=True)
output = layer_norm(torch.matmul(A, x))
loss = torch.dot(torch.tensor([0.1, 0.2, 0.5]), output) # torch.sum(torch.square(output))
loss.backward()
print(loss)
print(A.grad)

tensor(0.4314, grad_fn=<DotBackward>)
tensor([[ 0.1264,  0.1806,  0.1987],
        [-0.5067, -0.7239, -0.7963],
        [ 0.3803,  0.5433,  0.5976]])


In [86]:
data = np.array([0.1, 0.4, 0.5], dtype=np.float32) - 100.0
x = torch.tensor(data)
A = torch.eye(len(data), requires_grad=True)
z = torch.matmul(A, x)
z.retain_grad()
mu = torch.mean(z)
mu.retain_grad()
sigma = torch.sqrt(torch.std(z, unbiased=False) ** 2)
sigma.retain_grad()
o = (z - mu) / sigma
o.retain_grad()
l = torch.dot(torch.tensor([0.1, 0.2, 0.5]), o)
# l.backward()
# print(A.grad)
l.backward()
print(A.grad)

tensor([[-18.0826, -18.0283, -18.0102],
        [ 72.3376,  72.1204,  72.0480],
        [-54.2588, -54.0958, -54.0415]])


# Get norms and gradients for different models

In [35]:
model = clip_model.ClipModel('ViT-L/14')


In [24]:
model = clip_model.ClipModel('ViT-B/16')
model.new_last_layer(2)

In [41]:
# print(list(model.get_layers()[0][1].parameters())[0].requires_grad)
# inputs = torch.zeros((1, 3, 224, 224))

def get_clip_mean_input():
    inputs = torch.tensor([0.48145466, 0.4578275, 0.40821073])
    inputs = inputs.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
    inputs = inputs.tile([1, 1, 224, 224])
    return inputs

def get_clip_dist_input():
    inputs = torch.tensor([0.48145466, 0.4578275, 0.40821073])
    inputs = inputs.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
    inputs = inputs.tile([1, 1, 224, 224])
    return inputs

inputs = torch.tensor([0.48145466, 0.4578275, 0.40821073])
inputs = inputs.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
inputs = inputs.tile([1, 1, 224, 224])
print(inputs.shape)
inputs = inputs.cuda()
model.cuda()
outputs = model(inputs)
loss = torch.sum(torch.square(outputs))
loss.backward()

torch.Size([1, 3, 224, 224])


In [42]:

def get_grad_layer(cur_layer):
    grads = [p.grad.detach().cpu().numpy() for p in cur_layer.parameters()]
    grad_norms_squared = [np.linalg.norm(g) ** 2 for g in grads]
    grad_norm = np.sqrt(np.sum(grad_norms_squared))
    return grad_norm

def get_norm_layer(cur_layer):
    norms_squared = [np.linalg.norm(p.data.detach().cpu().numpy()) ** 2 for p in cur_layer.parameters()]
    norm = np.sqrt(np.sum(norms_squared))
    return norm

def get_layer_grads(model):
    named_layers = model.get_layers()
    names, layers = zip(*named_layers)
    norms = [get_norm_layer(l) for l in layers]
    grad_norms = [get_grad_layer(l) for l in layers]
    return norms, grad_norms

norms, grad_norms = get_layer_grads(model)
print(grad_norms)
    

[29.54964256286621, 2.6062560390040383, 7.567190647125244, 1.0375860929489136, 0.26622636570179553, 9.328513982084095, 0.2186721176275342, 17.19362955770947, 0.26272152142902877, 9.570395973223832, 0.33883179052500684, 17.063759606866164, 0.4135279331824097, 19.316852593622446, 0.348065239809437, 23.211175772550735, 0.2866482939208723, 21.411272157777947, 0.2982341959606744, 32.02183701601188, 0.7350253231739102, 23.98497888947061, 0.3489564675414618, 36.33611495543812, 0.3906924973175038, 26.982765200703604, 0.4056054973737743, 39.959679978199176, 0.46362568149293487, 25.90123800988363, 0.2922789139876288, 40.1849475230448, 1.12168652906377, 28.275337317915774, 0.2797527143483323, 48.489088348578655, 0.5411435171208155, 21.4489124869876, 0.23895772899326645, 34.997931414576485, 1.7938492117508384, 21.208721582016143, 0.16792367077919937, 30.11186549644565, 2.420525364382956, 19.86338208946117, 0.4717152135407727, 27.1885166259501, 1.0686553663789327, 34.53155986991048, 0.2615572246974

# Test get layers for different models

In [4]:
# convnext

convnext = timm_model.TimmModel('convnext_base_in22k')


In [13]:
layers = convnext.get_layers()
print('num layers: ', len(layers))
total_count = 0
for i in range(len(layers)):
    num_params = utils.count_parameters(layers[i][1], trainable=False) + utils.count_parameters(layers[i][1], trainable=True)
    total_count += num_params
    print(f'no. of params in layer {i} ({layers[i][0]}): {num_params}')
print(total_count)

num layers:  7
no. of params in layer 0 (stem): 6528
no. of params in layer 1 (stage0): 415488
no. of params in layer 2 (stage1): 1748992
no. of params in layer 3 (stage2): 57950208
no. of params in layer 4 (stage3): 27443200
no. of params in layer 5 (post_norm): 2048
no. of params in layer 6 (head): 22387025
109953489


In [22]:
# Bit-resnet (get layers) for ResNet-50 and ResNet-101

resnet50_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R50x1.npz"
resnet50 = bit_resnet.BitResNet(model_name='BiT-M-R50x1', checkpoint_path=resnet50_checkpoint_path)
get_layers_freeze_test(resnet50)

resnet101_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R101x1.npz"
resnet101 = bit_resnet.BitResNet(model_name='BiT-M-R101x1', checkpoint_path=resnet101_checkpoint_path)
get_layers_freeze_test(resnet101)

num params before freezing:  68256659
('conv-block-0', Sequential(
  (unit01): PreActBottleneck(
    (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (relu): ReLU(inplace=True)
    (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (unit02): PreActBottleneck(
    (gn1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (conv1): StdConv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3)

In [57]:
# DINO
model = vit_model.VitModel(model_name='dino_vitb16')
model.new_last_layer(10)
get_layers_freeze_test(model)


Downloading: "https://github.com/facebookresearch/dino/archive/main.zip" to /sailhome/ananya/.cache/torch/hub/main.zip


num params before freezing:  85806346
('empty_ln_pre', Module())
54
1
num params after freezing 1: 85215754
2
num params after freezing 2: 85215754
54
num params after freezing 54: 0


In [3]:
# Timm ViT-S
model = vit_model.VitModel(model_name='timm.vit_small_patch16_224')
get_layers_freeze_test(model)


num params before freezing:  22050664
('empty_ln_pre', Module())
54
num params after freezing 1: 21755368
num params after freezing 2: 21755368
num params after freezing 54: 0


In [24]:
# Get model grads and norms for Timm ViT-S
layers = model.get_layers()

for name, layer in layers:
    norms_squared = [np.linalg.norm(p.data.detach().cpu().numpy()) ** 2 for p in layer.parameters()]
    norm = np.sqrt(np.sum(norms_squared))
    print(name, norm)

patch_embed 26.6946963982087
empty_ln_pre 0.0
pos_embed 114.1449203491211
cls_token 11.814652442932129
trans0_norm1 10.727939400443407
trans0_attn 51.44874777832921
trans0_norm2 28.292761646663003
trans0_mlp 76.65611477620195
trans1_norm1 11.67158919875855
trans1_attn 51.968497785989186
trans1_norm2 16.081294584238012
trans1_mlp 72.49136824958491
trans2_norm1 14.588461930309668
trans2_attn 47.43609999667546
trans2_norm2 16.750402005493847
trans2_mlp 73.4898046975493
trans3_norm1 17.127796509537827
trans3_attn 46.12080652841503
trans3_norm2 16.560562623359647
trans3_mlp 73.75539281539531
trans4_norm1 16.38613601236467
trans4_attn 45.396057461621595
trans4_norm2 15.56170805432499
trans4_mlp 74.04262678751516
trans5_norm1 15.666596993058972
trans5_attn 45.22580724285582
trans5_norm2 16.125041315399216
trans5_mlp 74.32087040062893
trans6_norm1 16.50691386925591
trans6_attn 44.89644589636653
trans6_norm2 16.870073778570866
trans6_mlp 78.20890488715406
trans7_norm1 16.66077125665377
trans7_a

In [3]:
# Conv-next
model = timm_model.TimmModel('convnext_base_in22k')
get_layers_freeze_test(model)

num params before freezing:  109953489
('stage0', ConvNeXtStage(
  (downsample): Identity()
  (blocks): Sequential(
    (0): ConvNeXtBlock(
      (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=512, out_features=128, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
    )
    (1): ConvNeXtBlock(
      (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=512

In [9]:
x = torch.zeros((8,3,224,224))
x = x.cuda()
model(x)

tensor([[-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        ...,
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625]],
       device='cuda:0')

In [10]:
print(model._model.head)

Sequential(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Identity())
  (norm): LayerNorm2d((1024,), eps=1e-06, elementwise_affine=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=1024, out_features=21841, bias=True)
)


# Supervised ResNet

In [32]:
model = imnet_resnet.ResNet50(pretrained=True)

In [33]:
model.get_layers()

[('stem',
  ModuleList(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )),
 ('layer1',
  Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1